In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [18]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import numpy as np

In [3]:
def date_time(table_cells):
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]


def booster_version(table_cells):
    out='.'.join([booster_version for i,booster_version in enumerate(table_cells.strings) if i%2==0][0:-1])
    return out
    
def landing_status(table_cells):
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass = unicodedata.normalize('NFKD', table_cells.text).strip()
    if mass:
        mass.find('kg')
        new_mass=mass[0:mass.find('kg') + 2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    if(row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
    
    col_name = ' '.join(row.contents)
    
    if not(col_name.strip().isdigit()):
        col_name = col_name.strip()
        return col_name

In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

In [5]:
response = requests.get(static_url)
response.status_code

200

In [6]:
soup = BeautifulSoup(response.text)
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

In [7]:
html_tables = soup.find_all("table")

In [8]:
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
<tr>
<th rowspan="2" scope="row" style="text-align:center;">1
</th>
<td>

In [9]:
column_names= []
ths = first_launch_table.find_all("th")
for th in ths:
    col = extract_column_from_header(th)
    if col is not None and len(col)>0:
        column_names.append(col)
        

In [10]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


In [11]:
launch_dict = dict.fromkeys(column_names)

del launch_dict['Date and time ( )']

launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [19]:
extracted_row = 0

for table_number,table in enumerate(soup.find_all('table','wikitable plainrowheaders collapsible')):
    for rows in table.find_all('tr'):
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False

        row = rows.find_all('td')
        if flag:
            extracted_row += 1
            datatimelist = date_time(row[0])
            date = datatimelist[0].strip(',')
            time = datatimelist[1]

            bv = booster_version(row[1])
            if not(bv):
                bv = row[1].a.string
            print(bv)

            launch_site = row[2].a.string
            payload = row[3].a.string
            payload_mass = get_mass(row[4])
            orbit = row[5].a.string
            customer = row[6].a.string if row[6].a else np.nan
            launch_outcome = list(row[7].strings)[0]
            booster_landing = landing_status(row[8])
            
            launch_dict['Flight No.'].append(flight_number)
            launch_dict['Launch site'].append(launch_site)
            launch_dict['Payload'].append(payload)
            launch_dict['Payload mass'].append(payload_mass)
            launch_dict['Orbit'].append(orbit)
            launch_dict['Customer'].append(customer)
            launch_dict['Launch outcome'].append(launch_outcome)
            launch_dict['Version Booster'].append(booster_version)
            launch_dict['Booster landing'].append(booster_landing)
            launch_dict['Date'].append(date)
            launch_dict['Time'].append(time)

F9 v1.0
F9 v1.0
F9 v1.0
F9 v1.0
F9 v1.0
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 FT
F9 v1.1
F9 FT
F9 FT
F9 FT
F9 FT
F9 FT
F9 FT
F9 FT
F9 FT
F9 FT
F9 FT
F9 FT
F9 FT
F9 FT
F9 FT
F9 FT
F9 FT
F9 FT
F9 B4
F9 FT
F9 B4
F9 B4
F9 FT
F9 B4
F9 FT
F9 FT
F9 B4
F9 FT
F9 FT
F9 B4
F9 B4
F9 B4
F9 B4
F9 B5
F9 B4
F9 B4
F9 B4
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5 ♺
F9 B5 ♺
F9 B5 ♺
F9 B5 ♺
F9 B5
F9 B5
F9 B5
F9 B5 ♺
F9 B5 ♺
F9 B5 ♺
F9 B5 ♺
F9 B5 ♺
F9 B5
F9 B5 ♺
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5


In [20]:
df = pd.DataFrame({key:pd.Series(value) for key,value in launch_dict.items()})

In [21]:
df.head()

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,<function booster_version at 0x7fb8a4b49900>,Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,NASA,Success,<function booster_version at 0x7fb8a4b49900>,Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,<function booster_version at 0x7fb8a4b49900>,No attempt\n,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,<function booster_version at 0x7fb8a4b49900>,No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,<function booster_version at 0x7fb8a4b49900>,No attempt\n,1 March 2013,15:10


In [22]:
df.isna().sum()

Flight No.         0
Launch site        0
Payload            0
Payload mass       0
Orbit              0
Customer           3
Launch outcome     0
Version Booster    0
Booster landing    0
Date               0
Time               0
dtype: int64

In [23]:
df.to_csv('spacex_web_scraped.csv', index=False)